In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
# 变换

nn.Embedding()
	# nn.Embedding(vocabulary_size, embedding_dimension).from_pretrained(vectors, freeze)
	# 默认freeze=True; 若为True, 则张量在学习过程中不会被更新. 等价于embedding.weight.requires_grad = False
	# .weight为可学习权重, 从标准正态分布初始化
	# embeddings.weight.data.copy_(vectors), 等价于embedding.weight.data[i] = vectors[token]
nn.Linear()
	# 参数bias默认为True
	# .weight/.bias
	# .weight.data.shape; .weight.data.copy_()
	# nn.Linear(20, 30), .weight的shape为30,20
	# 不要使用太大的线性层, 因为nn.Linear(m,n)使用的是的O(mn)的内存, 线性层太大很容易超出现有显存

In [ ]:
# 卷积

# torch.nn.Conv2d(channels, output, height_of_filter, width_of_filter) 
x = torch.randn(2,1,7,3) # 输入的形状[batch_size, channels, height_of_image, width_of_image]
model = torch.nn.Conv2d(1,8,(2,3)) # height_of_filter, width_of_filter也可由一个值kernel_size给出; stride步长参数也可由一个元组或一个值给出
res = model(x)
print(res.shape) # torch.Size([2, 8, 6, 1]), 输出的高和宽分别是height_of_image-height_of_filter+1, width_of_image-width_of_filter+1

input = torch.randn(20, 16, 50, 100)
m = nn.Conv2d(16, 33, 3, stride=2) 
print(m(input).shape)  # torch.Size([20, 33, 24, 49]), 输出的高和宽需要除以stride再+1
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2)) # padding是在输入的两边添加0
print(m(input).shape) # torch.Size([20, 33, 28, 100])
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1)) # dilation指核元素之间的Spacing, 默认为1, 如height_of_filter=(height_of_filter-1)*dilation+1
print(m(input).shape) # torch.Size([20, 33, 26, 100])

In [ ]:
# RNN

# nn.LSTM(input_size, hidden_size, num_layers=1, bias=True, batch_first=False, dropout=0, bidirectional=False)
# 输入为input, (h_0, c_0); 其中input形状为(seq_len, batch, input_size), h_0, c_0形状均为(num_layers * num_directions, batch, hidden_size)
# 输出为output, (h_n, c_n); 其中output形状为(seq_len, batch, num_directions * hidden_size), h_n, c_n形状同上
# 不要在太长的序列上使用RNN; 因为RNN反向传播使用的是BPTT算法, 其需要的内存和输入序列的长度呈线性关系
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
print(output.shape, hn.shape, cn.shape) # torch.Size([5, 3, 20]) torch.Size([2, 3, 20]) torch.Size([2, 3, 20])

# nn.utils.rnn.pack_padded_sequence
# x = pack_padded_sequence(x, x_len), 当数据集有长度为0的句子时, 会报错 
# nn.utils.rnn.pad_packed_sequence
seq = torch.tensor([[1,2,0], [3,0,0], [4,5,6]])
lens = [2, 1, 3]
packed = nn.utils.rnn.pack_padded_sequence(seq, lens, batch_first=True, enforce_sorted=False) 
# 压缩包含可变长度填充序列的张量
# 若enforce_sorted为True, 则lens必须是降序
'''
PackedSequence(data=tensor([4, 1, 3, 5, 2, 6]), batch_sizes=tensor([3, 2, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))
batch_sizes的长度就是mini_batch数据中句子的最大长度, 每个元素代表列的长度
sorted_indices中的索引为原始索引, 且排好序
unsorted_indices的索引为结果索引, 对应原始的顺序
'''
seq_unpacked, lens_unpacked = nn.utils.rnn.pad_packed_sequence(packed, batch_first=True) # 还有参数padding_value=0.0, total_length=None
# 填充一批包装好的可变长度序列
'''
(tensor([[1, 2, 0],
         [3, 0, 0],
         [4, 5, 6]]), tensor([2, 1, 3]))
'''

In [ ]:
# Transformer

# nn.TransformerEncoderLayer这个类是transformer encoder的组成部分, 代表encoder的一个层
# 而encoder就是将transformerEncoderLayer重复几层
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8) 
# d_model: 输入的特征维度
# 还有参数dim_feedforward(默认2048), dropout, activation(默认relu)
# 输入形状为(seqlenth x batch x dim)形式的数据

# nn.TransformerEncoder是transformer的encoder部分, 即将上述的encoder-layer作为参数输入初始化以后可以获得TransformerEncoder
encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
# num_layers指encoder-layer的数目; norm: 层normalization
# 输入形状为(seqlenth x batch x dim)形式的数据

In [ ]:
# 池化

# 提取重要信息的操作, 可以去掉不重要的信息，减少计算开销
nn.MaxPool2d() 
nn.AvgPool2d()
# 参数情况可类比nn.Conv2d
# 步长的默认值是kernel_size

F.avg_pool2d() # 数据是四维输入
	# input形状: (batch_size, channels, height, width)
F.max_pool1d() 
F.max_pool2d()
F.lp_pool2d()

In [ ]:
# 激活函数

nn.Softmax()
	# F.softmax(input, dim=1)  # 按行softmax, 行和为1
nn.LogSoftmax() # 对softmax的结果进行log, 即Log(Softmax(x))

nn.ReLU(inplace=True) # 修正线性单元(Rectified Linear Unit, 整流线性单位函数), 通常指代以斜坡函数f(x)=max(0,x)及其变种为代表的非线性函数
	# torch.relu()
	# F.relu()
	# F.softplus() # relu函数的平滑版本
nn.LeakyReLU(0.1)
nn.ELU()
nn.GELU()

nn.Sigmoid()
	# 逻辑函数(Logistic sigmoid)
	# torch.sigmoid() # 将变量映射到0,1之间, 不使用nn.functional.sigmoid()
	# F.sigmoid()
nn.Tanh() # 双曲函数
	# torch.tanh() # 图像为过原点并将变量映射到-1,1之间

In [ ]:
# 网络相关

nn.Module
# 除__init__函数, 还要实现forward函数(从输入到输出)
# module.weight.data, module.bias.data
# register_buffer(name: str, tensor: torch.Tensor, persistent: bool = True) -> None: 为module添加一个buffer
# register_parameter(name: str, param: torch.nn.parameter.Parameter) -> None: 为module添加一个parameter
nn.ModuleList()
nn.Sequential()
	# .add_module()
	# nn.Sequential(*layers) # layers为列表
	# model = nn.Sequential(nn.Linear(2, 2, bias=False))或model = nn.Sequential(nn.Linear(2, 2, bias=False), nn.Tanh()); 可使用model[0]访问线性网络
	# model.modules()
	# model的一些属性: training/bool, _parameters/OrderedDict(), _buffers/OrderedDict(), _modules/OrderedDict()
# 可以通过model.state_dict()或者model.named_parameters()函数查看现在的全部可训练参数(包括通过继承得到的父类中的参数)
params = list(model.named_parameters())
(name, param) = params[28]
print(name)
print(param.grad)
(name2, param2) = params[29]
print(name2)
print(param2.grad)
(name1, param1) = params[30]
print(name1)
print(param1.grad)

# 定义可训练的参数
nn.Parameter() # Variable的一种, 参数为data
# nn.Parameter(tensor, requires_grad)
# 与torch.tensor(list, requires_grad=True)的区别: 这个只是将参数变成可训练的, 并没有绑定在module的parameter列表中

nn.BatchNorm2d() # 参数是num_features, 默认是有可学习的参数, 若affine=False则无
nn.BatchNorm1d()
nn.LayerNorm(2, elementwise_affine=True)
nn.utils.clip_grad_norm_() # 梯度裁剪(gradient clipping), 第一个参数一般为model.parameters(), 第二个参数为max_norm
# 可以预防RNNs/LSTMs中的梯度爆炸问题
F.normalize() # L2 normalization, 第一个参数tensor, 第二个参数dim

dropout = 0
nn.Dropout(dropout)

pad = None
F.pad(input, pad, mode='constant', value=0) # 便于对数据集图像或中间层特征进行维度扩充
# pad: 扩充维度, 用于预先定义出某维度上的扩充参数
	# p1d = (1, 2)即(左边填充数, 右边填充数), 针对最后一维
	# p2d = (1, 2, 3, 4)即(左边填充数, 右边填充数, 上边填充数, 下边填充数), 针对最后两维
# mode: 扩充方法, 'constant', 'reflect'或'replicate'三种模式, 分别表示常量, 反射, 复制
# value: 扩充时指定补充值, 但是value只在mode='constant'有效, 即使用value填充在扩充出的新维度位置; 而在'reflect'和'replicate'模式下, value不可赋值

nn.CosineSimilarity(dim=2)

In [ ]:
# 损失函数/criterion

nn.MSELoss()

nn.BCELoss()
nn.BCEWithLogitsLoss() # BCEWithLogitsLoss是针对二分类的CrossEntropy

weight, reduction = None, None
nn.CrossEntropyLoss(weight, reduction='mean') # 输入不需要经过Softmax; 参数weight: 为每个类指定权重, reduction: 对输出应用, 'none', 'mean', 'sum', 默认为mean
# 等价于torch.nn.functional.log_softmax + torch.nn.NLLLoss
input, target, weight = None, None, None
F.cross_entropy(input, target, reduction='none', weight=weight) # weight确定每个类别的权重
# 该函数使用了log_softmax和nll_loss
F.nll_entropy()

In [ ]:
# 初始化

nn.init.constant_()
nn.init.uniform_()
nn.init.normal_()
nn.init.xavier_uniform_()
nn.init.xavier_normal_()
nn.init.kaiming_uniform_()
tensor = None
nn.init.orthogonal_(tensor, gain=1) # 使得tensor是正交的